In [1]:
import pandas as pd
import numpy as np
import pickle as cp
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Input, TimeDistributed, Reshape
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Activation, Dense, MaxPooling2D
import keras.backend as K
from keras.engine.topology import Layer
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [5]:
# import train data
adl_1_1 = pd.read_csv("ADL1Opportunity_locomotion_S1.csv",header=None)
adl_1_2 = pd.read_csv("ADL2Opportunity_locomotion_S1.csv",header=None)
adl_1_3 = pd.read_csv("ADL3Opportunity_locomotion_S1.csv",header=None)
adl_1_4 = pd.read_csv("ADL4Opportunity_locomotion_S1.csv",header=None)
adl_1_5 = pd.read_csv("ADL5Opportunity_locomotion_S1.csv",header=None)
drill_1 = pd.read_csv("Drill1Opportunity_locomotion.csv",header=None)
adl_2_1 = pd.read_csv("ADL1Opportunity_locomotion_S2.csv",header=None)
adl_2_2 = pd.read_csv("ADL2Opportunity_locomotion_S2.csv",header=None)
drill_2 = pd.read_csv("Drill2Opportunity_locomotion.csv",header=None)
adl_3_1 = pd.read_csv("ADL1Opportunity_locomotion_S3.csv",header=None)
adl_3_2 = pd.read_csv("ADL2Opportunity_locomotion_S3.csv",header=None)
drill_3 = pd.read_csv("Drill3Opportunity_locomotion.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("ADL3Opportunity_locomotion_S2.csv",header=None)
adl_3_3 = pd.read_csv("ADL3Opportunity_locomotion_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("ADL4Opportunity_locomotion_S3.csv",header=None)
adl_2_5 = pd.read_csv("ADL5Opportunity_locomotion_S3.csv",header=None)
adl_3_4 = pd.read_csv("ADL4Opportunity_locomotion_S3.csv",header=None)
adl_3_5 = pd.read_csv("ADL5Opportunity_locomotion_S3.csv",header=None)

In [6]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)
print('Shape of train data is {}'.format(train_data.shape))
print('Shape of test data is {}'.format(test_data.shape))
train_data.head()

Shape of train data is (495691, 114)
Shape of test data is (108352, 114)


,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,labels
0,148.0,956.0,-358.0,19.0,986.0,196.0,95.0,975.0,152.0,194.0,...,319.0,-845.0,-20.0,57.0,42.0,57.0,20.0,42.0,175.0,1
1,89.0,973.0,-287.0,10.0,1004.0,162.0,125.0,968.0,122.0,224.0,...,325.0,-847.0,-17.0,38.0,31.0,38.0,17.0,31.0,175.0,1
2,130.0,988.0,-418.0,-11.0,1014.0,202.0,127.0,1002.0,113.0,279.0,...,328.0,-852.0,27.0,31.0,15.0,31.0,-27.0,15.0,175.0,1
3,89.0,980.0,-425.0,-47.0,1025.0,191.0,110.0,1006.0,105.0,353.0,...,321.0,-852.0,26.0,22.0,-2.0,22.0,-26.0,-2.0,175.0,1
4,64.0,857.0,-391.0,-8.0,1022.0,204.0,97.0,1002.0,93.0,548.0,...,321.0,-850.0,22.0,45.0,-7.0,45.0,-22.0,-7.0,175.0,1


In [7]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [8]:
print(scaled_train.shape)
print(train_labels.shape)

(495691, 113)
(495691, 5)


In [9]:
num_sensors = 113
window_size = 30
step_size = 10
classes = 5

In [10]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [11]:
print(train_segments.shape)

(49567, 30, 113)


In [12]:
train_segments = train_segments.reshape((-1,num_sensors, window_size,1))
test_segments = test_segments.reshape((-1,num_sensors, window_size,1))

In [13]:
print(train_segments.shape)

(49567, 113, 30, 1)


In [14]:
from keras import backend as K

class ConcatenationLayer(Layer):

    def __init__(self, output_dim,**kwargs):
        self.output_dim = output_dim

        super(ConcatenationLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.concatenating_weights = self.add_weight(name='kernel', 
                                      shape=(113,20),
                                      initializer='uniform',
                                      trainable=True)
        

    def call(self, x):
    
       # for i in range(0,50):
        #    w[:,:,:,i]=tf.multiply(x[:,:,:,i],coeff_norm)
        x_reshaped=tf.concat(x,axis=1)
        
        next_layer=tf.tanh(tf.matmul(tf.matmul(x_reshaped,self.concatenating_weights),tf.ones(1,20)))
        

        return next_layer
    

    def compute_output_shape(self, input_shape):
         return output_dim
        
        
        

class NormalizationLayer(Layer):

    def __init__(self, output_dim,**kwargs):
        self.output_dim = output_dim

        super(NormalizationLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.alpha = self.add_weight(name='alpha', 
                                      shape=(1,1),
                                      initializer='uniform',
                                      trainable=True)
        self.beta = self.add_weight(name='beta', 
                                      shape=(1,1),
                                      initializer='uniform',
                                      trainable=True)
        self.k = self.add_weight(name='k', 
                                      shape=(1,1),
                                      initializer='uniform',
                                      trainable=True)
        super(NormalizationLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        layer_squared = K.square(x)
        coeff_norm=K.pow((self.k+self.alpha*K.sum(layer_squared, axis=3)),-self.beta)
        print(coeff_norm.shape)
        

        
      
       # for i in range(0,50):
        #    w[:,:,:,i]=tf.multiply(x[:,:,:,i],coeff_norm)
        x_unpacked = tf.unstack(x,axis=3)
        print(x_unpacked[0])
        processed=[]
        for t in x_unpacked:
        # do whatever
            result_tensor=tf.multiply(t,coeff_norm)
            
            processed.append(result_tensor)
        output = tf.stack(processed, axis=3) 
        print(output.shape)
        return output
    

    def compute_output_shape(self, input_shape):
         return input_shape

In [15]:
model = Sequential()
size_of_kernel_1_2 = (1,5)
size_of_kernel_3= (1,3)
kernel_strides = 1
num_filters_1_conv_layer = 50
num_filters_2_conv_layer = 40
num_filters_3_conv_layer = 20
dropout_prob = 0
inputshape = (num_sensors, window_size,1)

In [16]:
model.add(Conv2D(num_filters_1_conv_layer, kernel_size=size_of_kernel_1_2, strides=kernel_strides,
                 activation='tanh', input_shape=inputshape, name='1_conv_layer'))

model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=(1, 2), strides=None, padding='valid', data_format=None,name='maxpooling1'))
model.add(NormalizationLayer(output_dim=model.layers[-1].input_shape,name='normalization_layer1'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
(?, 113, 13)
Tensor("normalization_layer1/unstack:0", shape=(?, 113, 13), dtype=float32)
(?, 113, 13, 50)


In [17]:
model.add(Conv2D(num_filters_2_conv_layer, kernel_size=size_of_kernel_1_2, strides=kernel_strides,
                 activation='tanh', input_shape=inputshape, name='2_conv_layer'))


model.add(Activation('relu',name='relu2'))
model.add(MaxPooling2D(pool_size=(1, 3), strides=None, padding='valid', data_format=None,name='maxpooling2'))
model.add(NormalizationLayer(output_dim=model.layers[-1].input_shape,name='normalization_layer2'))

(?, 113, 3)
Tensor("normalization_layer2/unstack:0", shape=(?, 113, 3), dtype=float32)
(?, 113, 3, 40)


In [18]:
model.add(Conv2D(num_filters_3_conv_layer, kernel_size=size_of_kernel_3, strides=kernel_strides,
                 activation='tanh', input_shape=inputshape, name='3_conv_layer'))

model.add(Activation('relu',name='relu3'))

In [19]:

model.add(Flatten())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [20]:
model.add(Dense(units=400, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='1_dense_layer'))
model.add(Activation('relu',name='1_dense_relu'))

In [21]:
#model.add(Reshape((400,1)))

In [22]:
model.add(Dense(classes, activation='softmax',name='softmax_layer'))
rms = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [23]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

1_conv_layer: input shape: (None, 113, 30, 1) output shape: (None, 113, 26, 50)
relu1: input shape: (None, 113, 26, 50) output shape: (None, 113, 26, 50)
maxpooling1: input shape: (None, 113, 26, 50) output shape: (None, 113, 13, 50)
normalization_layer1: input shape: (None, 113, 13, 50) output shape: (None, 113, 13, 50)
2_conv_layer: input shape: (None, 113, 13, 50) output shape: (None, 113, 9, 40)
relu2: input shape: (None, 113, 9, 40) output shape: (None, 113, 9, 40)
maxpooling2: input shape: (None, 113, 9, 40) output shape: (None, 113, 3, 40)
normalization_layer2: input shape: (None, 113, 3, 40) output shape: (None, 113, 3, 40)
3_conv_layer: input shape: (None, 113, 3, 40) output shape: (None, 113, 1, 20)
relu3: input shape: (None, 113, 1, 20) output shape: (None, 113, 1, 20)
flatten_1: input shape: (None, 113, 1, 20) output shape: (None, 2260)
1_dense_layer: input shape: (None, 2260) output shape: (None, 400)
1_dense_relu: input shape: (None, 400) output shape: (None, 400)
softmax

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1_conv_layer (Conv2D)        (None, 113, 26, 50)       300       
_________________________________________________________________
relu1 (Activation)           (None, 113, 26, 50)       0         
_________________________________________________________________
maxpooling1 (MaxPooling2D)   (None, 113, 13, 50)       0         
_________________________________________________________________
normalization_layer1 (Normal (None, 113, 13, 50)       3         
_________________________________________________________________
2_conv_layer (Conv2D)        (None, 113, 9, 40)        10040     
_________________________________________________________________
relu2 (Activation)           (None, 113, 9, 40)        0         
_________________________________________________________________
maxpooling2 (MaxPooling2D)   (None, 113, 3, 40)        0         
__________

In [ ]:
batchSize = 100
train_epoches = 50
model.fit(train_segments,train_labels,validation_data=(test_segments,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(test_segments,test_labels,verbose=1)
print(score)
model.save('taskA_all_Subjects_DEEP_CNN_model.h5')

Train on 49567 samples, validate on 10833 samples
Epoch 1/50
49567/49567 [==============================] - 47s 945us/step - loss: 1.5162 - acc: 0.4224 - val_loss: 1.5035 - val_acc: 0.3582
Epoch 2/50
49567/49567 [==============================] - 42s 840us/step - loss: 1.4179 - acc: 0.4231 - val_loss: 1.4908 - val_acc: 0.3582
Epoch 3/50
49567/49567 [==============================] - 42s 848us/step - loss: 1.3877 - acc: 0.4231 - val_loss: 1.4981 - val_acc: 0.3582
Epoch 4/50
49567/49567 [==============================] - 42s 849us/step - loss: 1.3792 - acc: 0.4231 - val_loss: 1.5074 - val_acc: 0.3582
Epoch 5/50
49567/49567 [==============================] - 42s 850us/step - loss: 1.3773 - acc: 0.4231 - val_loss: 1.5131 - val_acc: 0.3582
Epoch 6/50
49567/49567 [==============================] - 42s 857us/step - loss: 1.3769 - acc: 0.4231 - val_loss: 1.5156 - val_acc: 0.3582
Epoch 7/50
49567/49567 [==============================] - 42s 855us/step - loss: 1.3768 - acc: 0.4231 - val_loss: 1.

In [ ]:
predictions = model.predict(test_segments)

In [ ]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)